# Travel

(Last updated: May 29, 2024)

After traveling to various cultural sites for some years, I found a rule of thumb for finding good spots to visit: check if something is a UNESCO World Heritage site. Sometimes this leads to obvious suggestions (Vatican City, Paris, Amsterdam) but more often than not visiting such sites has shown me sites that are very much worth visiting but not as touristy as the usual sites (e.g., Bologna, Orange, Albi).

At first, my enthousiasm led to the dream of visiting all world heritage sites in Europe, but this dream soon showed to be too ambitious. All EU countries then? Still quite a lot. Some narrowing down brought me to the following goal: visit all world heritage sites in the continental part of the original member states of the European Union. Achievable, but also ambitious, with Germany, France and Italy as the biggest obstacles. Since then, I've been using this as a heuristic in planning vacations.

(Of course, I still travel outside of these guidelines as well. Occasionally this leads to another world heritage site, and these are also included in the below map)



In [9]:
import pandas as pd
import plotly.express as px
import numpy as np

df = pd.read_csv("whc-sites-2023.csv")
df = df[df['Visited'].notna()]
max_bound = max(abs(min(df['latitude'])-max(df['latitude'])), abs(min(df['longitude'])-max(df['longitude']))) * 175
zoom = 11.5 - np.log(max_bound)
fig = px.scatter_mapbox(df, lat='latitude', lon='longitude', color='Visited',
                        hover_name='name_en', mapbox_style='carto-positron', zoom=zoom,
                        hover_data={
                            'latitude':False,
                            'longitude':False
                        })
fig.update_layout(showlegend=False, margin={'t':0,'l':0,'b':0,'r':0})
fig.show()


In [10]:
print("Progress:")
score = len(df[(df['Visited'])]) / len(df)
displays = int(score * 40)
print(f"Total: {" " * (20 - len("Total"))}[{"+" * displays}{"." * (40 - displays)}] {len(df[(df['Visited'])])} out of {len(df)}\n")
for abbr, country in zip(['nld', 'fra', 'ita', 'bel', 'deu', 'lux'], ["Netherlands", "France", "Italy", "Belgium", "Germany", "Luxembourg"]):
    score = len(df[(df['Visited']) & (df['udnp_code'].str.contains(abbr))]) / len(df[df['udnp_code'].str.contains(abbr)])
    displays = int(score * 40)
    print(f"{country}: {" " * (20 - len(country))}[{"+" * displays}{"." * (40 - displays)}] {len(df[(df['Visited']) & (df['udnp_code'].str.contains(abbr))])} out of {len(df[df['udnp_code'].str.contains(abbr)])}")

Progress:
Total:                [++++++++++..............................] 49 out of 185

Netherlands:          [+++++++++++++...........................] 4 out of 12
France:               [+++++++++++.............................] 13 out of 47
Italy:                [+++++...................................] 8 out of 57
Belgium:              [++++++++++..............................] 4 out of 16
Germany:              [++++....................................] 6 out of 51
Luxembourg:           [........................................] 0 out of 1
